In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import numpy as np

from model.distance_func import accident_around_road

# 合併事故資料以及路口資料

In [3]:
taipei_road = pd.read_excel('DATA/路口特徵with_7-11&youbike.xlsx')

unique_accidence = pd.read_csv('DATA/111_taipei_accidence_unique_with_dummies.csv')

找到事故發生地 100 公尺內最近的路口 \
程式碼請參考
[事故路口](./model/distance_func.py)

In [5]:
aar = accident_around_road(taipei_road, unique_accidence, 100)

In [6]:
road_indicies = np.where(aar.transpose() == 1)

找到最近的路口後將路口編號合併進事故資料

In [7]:
unique_accidence.loc[road_indicies[0], 'road_index'] = list(road_indicies[1])
unique_accidence = unique_accidence[pd.notna(unique_accidence['road_index'])]

將事故資料依照發生時間分成上午及下午，並將下午資料的路口編號加上總路口數，以便後續整合資料

In [11]:
# 將 7-13、13-19 分成上午跟下午
unique_accidence['morning'] = np.where((unique_accidence['發生時'] >= 7) & (unique_accidence['發生時'] <= 12), 1, 0)
# 區成上午跟下午後，為了要 map 回路口的 dataframe，需要將下午資料的 road_index + 257

row_num_of_road = len(taipei_road)
unique_accidence.loc[unique_accidence['morning'] == 0, 'road_index'] += row_num_of_road

將上下午車流量資料分開處理並合併

In [14]:
# 將車流量資料區分成上午跟下午 : 使用 copy 一個資料後用 concat 的方式
taipei_road_afternoon = taipei_road.copy()

# 原始資料只留下早上的流量
taipei_road = taipei_road.drop(columns = ['afternoon_truck', 'afternoon_car',	'afternoon_scooter'])

taipei_road.rename(columns = {'morning_truck':'truck_flow', 'morning_car': 'car_flow', 
                              'morning_scooter': 'scooter_flow'}, inplace = True)

# 複製後的資料留下下午的流量
taipei_road_afternoon = taipei_road_afternoon.drop(columns = ['morning_truck', 'morning_car', 'morning_scooter'])

taipei_road_afternoon.rename(columns = {'afternoon_truck':'truck_flow', 'afternoon_car': 'car_flow', 
                              'afternoon_scooter': 'scooter_flow'}, inplace = True)

taipei_road = pd.concat([taipei_road, taipei_road_afternoon], ignore_index=True)

計算每個路口早上、下午分別發生幾場交通事故

In [15]:
accident_num = pd.DataFrame({'count' : unique_accidence.groupby( ['road_index', 'morning'] ).size()}).reset_index()

accident_num

,road_index,morning,count
0,0.0,1,1
1,1.0,1,1
2,2.0,1,4
3,6.0,1,3
4,8.0,1,1
...,...,...,...
481,509.0,0,2
482,510.0,0,6
483,511.0,0,7
484,512.0,0,15


在路口資料新增一個欄位 [accidence_num]，用以儲存計算結果

In [ ]:
taipei_road.loc[accident_num['road_index'].astype(int), 'accidence_num'] = list(accident_num['count'])

In [18]:
taipei_road

,road_id,road_name,location,road_query,truck_flow,car_flow,scooter_flow,Y,X,separate_left_turn_num,no_left_turn_num,sidewalk,retracted_crosswalk,waiting_area,clear_sign,count_711,count_youbike,accidence_num
0,A001,大度路,大度路,大度路,180.6100,4461.7600,3944.3300,25.122283,121.481005,0,0,0,0,0,0.0,0,0,1.0
1,A002,重陽橋,重陽橋,重陽橋,49.7400,3317.0100,5027.5800,25.085503,121.504692,0,0,0,0,0,0.0,3,5,1.0
2,A003,臺北橋,臺北橋,臺北橋,45.6452,1478.4366,8367.6220,25.063505,121.507331,0,0,0,0,0,0.0,3,5,4.0
3,A004,忠孝橋,忠孝橋,忠孝橋,121.1334,4187.8408,4742.2824,25.051883,121.500685,0,0,0,0,0,0.0,0,0,NaN
4,A005,中興橋,中興橋,中興橋,87.0188,2406.6544,4063.9086,25.047398,121.495751,0,0,0,0,0,0.0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,SI045,木柵路~新光路,"['木柵路五段', '新光路二段', '木柵路五段', '-']",木柵路五段 新光路二段,100.9900,477.1500,466.8600,25.002947,121.583968,1,1,1,1,0,1.0,0,0,2.0
510,SI046,萬大路~西藏路,"['西藏路', '萬大路', '西藏路', '萬大路']",西藏路 萬大路,87.1900,724.9900,1960.8200,25.029447,121.500570,4,0,1,1,1,1.0,8,11,6.0
511,SI047,萬大路~東園街,"['萬大路423巷', '萬大路', '東園街', '萬大路']",萬大路423巷 萬大路,94.1500,716.6400,1590.2100,25.021520,121.498698,0,2,1,1,1,0.0,8,11,7.0
512,SI048,松高路~松仁路,"['松高路', '松仁路', '松高路', '松仁路']",松高路 松仁路,118.7500,1714.0000,1488.2500,25.039069,121.568518,2,0,1,1,1,1.0,12,12,15.0


將含有虛擬變數的事故資料依照路口編號進行分組，並取平均後與路口資料合併

In [13]:
specific_columns = ['暴雨', '霧或煙', '雨', '陰', '晴', '日間自然光線', '晨或暮光', 
                    '夜間(或隧道、地下道、涵洞)有照明', '夜間(或隧道、地下道、涵洞)無照明', 
                    '道路型態_三岔路', '道路型態_四岔路', '道路型態_多岔路', '道路型態_隧道', 
                    '道路型態_地下道', '道路型態_橋樑', '道路型態_涵洞', '道路型態_高架道路', 
                    '道路型態_彎曲路及附近', '道路型態_坡路', '道路型態_巷弄', '道路型態_直路', 
                    '道路型態_其他單路', '道路型態_圓環', '道路型態_廣場', '事故位置_交岔路口內', 
                    '事故位置_交岔口附近', '事故位置_機車待轉區', '事故位置_機車停等區', 
                    '事故位置_交通島_含槽化線', '事故位置_迴轉道', '事故位置_快車道', '事故位置_慢車道', 
                    '事故位置_一般車道_未劃分快慢車道', '事故位置_公車專用道', '事故位置_機車專用道', 
                    '事故位置_機車優先道', '事故位置_路肩、路緣', '事故位置_加速車道', '事故位置_直線匝道', 
                    '事故位置_環道匝道', '事故位置_行人穿越道', '事故位置_穿越道附近', '事故位置_人行道', 
                    '路面鋪裝_柏油', '路面鋪裝_水泥', '路面鋪裝_碎石', '路面鋪裝_其他鋪裝', '無鋪裝', 
                    '路面狀態_油滑', '路面狀態_泥濘', '路面狀態_濕潤', '路面狀態_乾燥', '路面缺陷_路面鬆軟', 
                    '路面缺陷_突出(高低)不平', '路面缺陷_有坑洞', '無缺陷', '障礙物_道路工程', '障礙物_有堆積物', 
                    '障礙物_路上有停車', '障礙物_其他障礙物', '無障礙物', '視距_彎道', '視距_坡道', '視距_建築物', 
                    '視距_樹木、農作物', '視距_路上停放車輛', '視距_其他', '視距_良好', '行車管制號誌', 
                    '行車管制號誌(附設行人專用號誌)', '閃光號誌', '無號誌', '號誌正常', '號誌不正常', 
                    '號誌無動作', '無號誌_無號誌', '分向設施_中央分向島_寬式', '分向設施_中央分向島_窄式附柵欄', 
                    '分向設施_中央分向島_窄式無柵欄', '分向設施_雙向禁止超車線_附標記', 
                    '分向設施_雙向禁止超車線_附標記.1', '分向設施_單向禁止超車線_附標記', 
                    '分向設施_單向禁止超車線_無標記', '分向設施_行車分向線_附標記', '分向設施_行車分向線_無標記', 
                    '分向設施_無分向設施', '分道設施_禁止變換車道線_附標記', '分道設施_禁止變換車道線_無標記', 
                    '分道設施_車道線_附標記', '分道設施_車道線_無標記', '分道設施_未繪設車道線', 
                    '分道設施_寬式快慢車道分隔島', '分道設施_窄式快慢車道分隔島_附柵欄', 
                    '分道設施_窄式快慢車道分隔島_無柵欄', '分道設施_快慢車道分隔線', 
                    '分道設施_未繪設快慢車道分隔線', '分道設施_有路面邊線', '分道設施_無路面邊線', 
                    '事故類型_人與汽(機)車_對向通行中', '事故類型_人與汽(機)車_同向通行中', 
                    '事故類型_人與汽(機)車_穿越道路中', '事故類型_人與汽(機)車_在路上嬉戲', 
                    '事故類型_人與汽(機)車_在路上作業中', '事故類型_人與汽(機)車_衝進路中', 
                    '事故類型_人與汽(機)車_從停車後(或中)穿出', '事故類型_人與汽(機)車_佇立路邊(外)', 
                    '事故類型_人與汽(機)車_其他', '事故類型_車與車_對撞', '事故類型_車與車_對向擦撞', 
                    '事故類型_車與車_同向擦撞', '事故類型_車與車_追撞', '事故類型_車與車_倒車撞', 
                    '事故類型_車與車_路口交岔撞', '事故類型_車與車_側撞', '事故類型_車與車_其他', 
                    '事故類型_汽(機)車本身_路上翻車', '事故類型_汽(機)車本身_衝出路外', 
                    '事故類型_汽(機)車本身_撞護欄', '事故類型_汽(機)車本身_撞號誌', 
                    '事故類型_汽(機)車本身_撞交通島', '事故類型_汽(機)車本身_撞非固定設施', 
                    '事故類型_汽(機)車本身_撞橋梁、建築物', '事故類型_汽(機)車本身_撞路樹、電桿', 
                    '事故類型_汽(機)車本身_撞動物', '事故類型_汽(機)車本身_撞工程施工', 
                    '事故類型_汽(機)車本身_其他', 'morning']

accident_groupby = unique_accidence.groupby('road_index')[specific_columns].mean().reset_index()

In [19]:
taipei_road_with_dummies = pd.merge(taipei_road, accident_groupby, left_index=True, right_on='road_index')

把後續分析用不到的欄位刪掉

In [21]:
taipei_road_with_dummies = taipei_road_with_dummies.drop(columns = ['road_name', 'location', 'road_query', 'road_index', 'Y', 'X'])

In [22]:
# taipei_road_with_dummies.to_csv('taipei_road_with_dummies.csv')
taipei_road_with_dummies.to_csv('taipei_road_with_dummies_100m_500m.csv')

# 下一步

接著，我將利用 STATA 的 Double-Lasso 分析交通事故發生次數與交通流量間因果關係。\
請參考
[因果關係分析](./Step6_Double-Lasso.ipynb)